In [ ]:
!pip install keras_cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import cifar100
from keras_cv.layers import RandomShear
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model
import keras_cv
import os
import tempfile

Using TensorFlow backend


In [ ]:
n_epoch = 1000 # Il y a le early stopping
batch_size = 400


(X_train, y_train), raw_test = cifar100.load_data()
X_test, y_test = raw_test
raw_test = tf.data.Dataset.from_tensor_slices(raw_test)
X_train = np.float64(X_train)/255
X_test = np.float64(X_test)/255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = 100

In [ ]:
modele_enseignant = tf.keras.models.load_model("/content/drive/MyDrive/projet_cnam/modele_745.keras")
modele_enseigne = tf.keras.models.load_model("/content/drive/MyDrive/projet_cnam/modele_enseigne_2.keras")

In [ ]:
y_train_enseignant = modele_enseignant.predict(X_train)

1563/1563 [==============================] - 29s 17ms/step


In [ ]:
!pip install tensorflow_model_optimization
import tensorflow_model_optimization as tfmot
def apply_pruning_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.sparsity.keras.prune_low_magnitude(layer)
  return layer
model_for_pruning = tf.keras.models.clone_model(
    modele_enseigne,
    clone_function=apply_pruning_to_dense,
)


In [ ]:
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

model_for_pruning.compile(optimizer='adam', loss='kl_divergence', metrics=['accuracy'])

model_for_pruning.fit(
    X_train,
    y_train_enseignant,
    callbacks=callbacks,
    epochs=2,
)


Epoch 1/2
1563/1563 [==============================] - 23s 10ms/step - loss: 1.9149 - accuracy: 0.3818
Epoch 2/2
1563/1563 [==============================] - 15s 9ms/step - loss: 1.9066 - accuracy: 0.3845


In [ ]:
modele_enseigne.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.2239 - accuracy: 0.4159


[2.223886489868164, 0.41589999198913574]

In [ ]:
model_for_pruning.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.2239 - accuracy: 0.4159


[2.223886489868164, 0.41589999198913574]

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)


<ipython-input-14-11ad47b980ba>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: /tmp/tmpfu09hwmk.h5


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
pruned_tflite_model = converter.convert()

pruned_tflite_file = "/content/drive/MyDrive/projet_cnam/modele_prune.tflite"
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)


Saved pruned TFLite model to: /content/drive/MyDrive/projet_cnam/modele_prune.tflite


In [ ]:
from tqdm import tqdm
def format_image(image, label):
    image = tf.image.resize(image, (32, 32)) / 255.0
    return  image, label

test_batches = raw_test.map(format_image).batch(1)

interpreter = tf.lite.Interpreter(model_path=pruned_tflite_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

predictions = []

# This will report how many iterations per second, where each
# iteration is 100 predictions
test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(1000)):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))

    test_labels.append(label.numpy()[0])
    test_imgs.append(img)


# This will tell you how many of the predictions were correct
score = 0
for item in range(0,len(predictions)):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1

print("Out of 1000 predictions I got " + str(score) + " correct")

100%|██████████| 1000/1000 [00:08<00:00, 115.75it/s]

Out of 1000 predictions I got 421 correct


<_TakeDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.int64, name=None))>